###### NAME: JANMEJAY MOHANTY <br> COURSE: BIA 660 WS <br> COURSE NAME: WEB MINING <br> CWID: 20009315 <br> EMAIL: JMOHANTY@STEVENS.EDU <br> PROGRAM: ROTTEN TOMATOES SCRAPING

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import csv,re,time

In [2]:
# Extracts all the review data from the current page and writes to the specified writer
def get_data(writer:csv.writer):

    reviews = driver.find_elements(by=By.CSS_SELECTOR,value='div[class="row review_table_row"]')    #get all review divs in the current page

    for review in reviews:  # for each review div
       
        reviewer_name,review_date,review_text,review_polarity='NA','NA','NA','NA' # initialize critic, text, date, platform and rating
        
        try:
            # try to get the reviewer name
            reviewer_name_chunk=review.find_element(by=By.CSS_SELECTOR,value='a')   # get the reviewer name box
            reviewer_name=reviewer_name_chunk.text   # get the reviewer name string

        except NoSuchElementException as e: # reviewer name could not be found
            print('could not extract reviewer name')    
        
        try:    
            # try to get the date
            date_box=review.find_element(by=By.CSS_SELECTOR,value='div[class="review-date subtle small"]')  # get the date box
            review_date=date_box.text   # get the date string

        except NoSuchElementException as e: # review date could not be found
            print('could not extract review date')

        try:
            # try to get the review text
            review_text_chunk=review.find_element(by=By.CSS_SELECTOR,value='div[class="the_review"]')   # get the review text box
            review_text=review_text_chunk.text  # get the review text box string
        
        except NoSuchElementException as e: # review text could not be found
             print('could not extract review text')   
        
        try:
            # try to get the review polarity
            review_polarity_chunk = review.find_element(by=By.CSS_SELECTOR,value='div[class*="review_icon icon small"]')   # get the review polarity box
            review_polarity_value = review_polarity_chunk.get_attribute('class')
            if (review_polarity_value.find('rotten')>0):
                review_polarity = 'rotten'
            if (review_polarity_value.find('fresh')>0):
                review_polarity= 'fresh'  

        except NoSuchElementException as e: # review polarity could not be found
            print('could not extract review polarity')

        writer.writerow([reviewer_name,review_date,review_text,review_polarity]) # write a new row to file 


In [3]:
def scrape(driver,url):
    delay:int = 5   # number of seconds to wait 

    driver.maximize_window()
    driver.get(url)  # visit the page

    input() # wait until we can click on the cookie overlay

    page_count = 1  # keep track of page count

    # create a new csv writer for the review data
    fw=open('Output.csv','w',encoding="utf-8")  # output file

    writer=csv.writer(fw,lineterminator='\n')   # create a csv writer for this file
    
    writer.writerow(["reviewer_name", "review date", "review text", "review_polarity"])

    while True: # keep going until there are no more pages

        print("Page No: ",page_count)   # print current page count

        page_count+=1   # increment page count

        get_data(writer)    # extract and write the review data from the current page
        
        try:
            # wait until the next button appears
            next_button = WebDriverWait(driver, delay).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[class*="js-prev-next-paging-next btn prev-next-paging__button prev-next-paging__button-right"]')))
        
        except TimeoutException:
            break

        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(delay)
        
    fw.close()    

    

In [4]:
url = 'https://www.rottentomatoes.com/m/tenet/reviews'
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install())) 

In [5]:
scrape(driver,url)

Page No:  1
Page No:  2
Page No:  3
Page No:  4
Page No:  5
Page No:  6
Page No:  7
Page No:  8
Page No:  9
Page No:  10
Page No:  11
Page No:  12
Page No:  13
Page No:  14
Page No:  15
Page No:  16
Page No:  17
Page No:  18
Page No:  19
